In [93]:
from checkreq import FindImport, FindreqLibraries, changereqfile
from utils import *

In [94]:
list_of_libs = []

for dir in GetFilesInDirectory('src'):
    list_of_libs += FindImport(dir)

In [95]:
list_of_libs

['langchain', 'json', 'boto3', 'streamlit']

In [96]:
for dir in GetFilesInDirectory('src'):
    changereqfile("requirements2.txt", 'requirements.txt', dir)
    print(FindreqLibraries("requirements.txt", dir))

['langchain==0.1.16', 'langchain-aws==0.1.2', 'langchain-cohere==0.1.5', 'langchain-community==0.0.34', 'langchain-core==0.1.46', 'langchain-openai==0.0.2.post1', 'langchain-text-splitters==0.0.1', 'json5 @ file:///tmp/build/80754af9/json5_1624432770122/work', 'jsonpatch @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_3ajyoz8zoj/croot/jsonpatch_1714483362270/work', 'jsonpointer==2.1', 'jsonref==1.1.0', 'jsonschema @ file:///Users/builder/cbouss/perseverance-python-buildout/croot/jsonschema_1701803145350/work', 'jsonschema-specifications @ file:///Users/builder/cbouss/perseverance-python-buildout/croot/jsonschema-specifications_1701803122948/work', 'boto3==1.34.84', 'streamlit==1.33.0', 'streamlit-chat==0.1.1', 'streamlit-elements==0.1.0', 'streamlit_folium==0.19.1']


In [74]:
import dspy
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from typing import Annotated, Any, Dict, Optional, Sequence, TypedDict, List, Tuple

In [75]:
#Uses aws credentials from aws cli
bedrock = dspy.Bedrock(region_name="us-east-1")

model_kwargs =  { 
        "max_tokens": 4000,
        "temperature": 0.4,
        # "top_k": 50,
        # "top_p": 0.9
    }

lm = dspy.AWSMistral(bedrock, "mistral.mistral-large-2402-v1:0", **model_kwargs)

In [76]:
dspy.configure(lm=lm)

In [77]:
class Input(BaseModel):
    python_version: str = Field(description="Python version to be used")
    type_of_application: str = Field(description="Type of python application")
    work_directory: str = Field(description="Working directory of the application")
    entrypoint_filename: str = Field(description="File name for starting the application endpoint")

class Output(BaseModel):
    dockerfile: str = Field(description="The complete dockerfile code")


class CreateDockerFile(dspy.Signature):
    """I need a Dockerfile for a basic Streamlit app. The app is written in Python."""

    input: Input = dspy.InputField()
    output: Output = dspy.OutputField()


In [78]:
def detect_application(requirements_file, directory_structure):
    # Read the requirements.txt file
    with open(requirements_file, 'r') as file:
        requirements = file.read()

    # Get the Python version
    python_version = platform.python_version()

    # Determine the type of application based on dependencies
    type_of_application = None
    if 'streamlit' in requirements:
        type_of_application = 'Streamlit'
    elif 'langchain' in requirements:
        type_of_application = 'Langchain'
    elif 'Flask' in requirements:
        type_of_application = 'Flask'
    
    # Look for a common entry point file (app.py or main.py) in the directory structure
    entrypoint_filename = 'app.py'  # Default guess
    for file in directory_structure:
        if file.endswith(('app.py', 'main.py')):
            entrypoint_filename = os.path.basename(file)
            break
    
    # # Set the working directory
    # work_directory = 'src'  # Default work directory
    # for file in directory_structure:
    #     if 'src/' in file:
    #         work_directory = 'src'
    #         break

    class Input(BaseModel):
        dir_struct: List[str] = Field(description="List of all the files in the directory.")
       

    class Output(BaseModel):
        work_directory: str = Field(description="The working directory of the application.")
        entrypoint_filename: str = Field(description="The filename that is the entrypoint for the application.")


    class FindWorkingDirectory(dspy.Signature):
        f"""I need the working directory for this {type_of_application} application and also the entrypoint filename for running it so that I can create a Dockerfile."""

        input: Input = dspy.InputField()
        output: Output = dspy.OutputField()

    predictor = dspy.TypedPredictor(FindWorkingDirectory)
    input = Input(
        dir_struct = directory_structure,
    )
    mapped_parameters = predictor(input=input)
    print(mapped_parameters.output.work_directory)
    print(mapped_parameters.output.entrypoint_filename)
    
    final_output = {
        'python_version': python_version,
        'type_of_application': type_of_application,
        'work_directory': mapped_parameters.output.work_directory,
        'entrypoint_filename': mapped_parameters.output.entrypoint_filename
    }

    return final_output


output = detect_application('requirements2.txt', GetFilesInDirectory("."))
output

./
checkreq.py


{'python_version': '3.11.9',
 'type_of_application': 'Streamlit',
 'work_directory': './',
 'entrypoint_filename': 'checkreq.py'}

In [79]:
input = Input(
    python_version = output["python_version"],
    type_of_application = output["type_of_application"],
    work_directory = output["work_directory"],
    entrypoint_filename = output["entrypoint_filename"]
)

In [80]:
predictor = dspy.TypedPredictor(CreateDockerFile)

mapped_parameters = predictor(input=input)
print(mapped_parameters.output.dockerfile)

# Use an official Python runtime as a parent image
FROM python:3.11.9-slim

# Set the working directory in the container
WORKDIR /app

# Copy the current directory contents into the container at /app
COPY . /app

# Install any needed packages specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Make port 8501 available to the world outside this container
EXPOSE 8501

# Run app.py when the container launches
CMD ["streamlit", "run", "checkreq.py", "--server.port=8501", "--server.address=0.0.0.0"]


In [81]:
with open("Dockerfile", "w") as file:
    file.write(mapped_parameters.output.dockerfile)